# Overview

This notebook handles generating and exporting the tax maps for the year. Rather than using a *Map Series* in a Layout, which can be rather limited in its functionality, we are using direct access to the `arcpy.mp` module and preconfigured layout pages.

By combining this with some spatial selections and definition queries, we can generate tax maps for any given tax year.

## Parts of the Project

### Maps

|Map|Description|
|-|-|
|Township Locator|Highlights the township the tax map is in|
|Section Locator|Shows the sections of the township, with the tax map's section highlighted|
|Tax Map|Shows cadastral features and other reference layers|
|Page Locator|*For 100 Scale Pages Only*; Shows the individual page w/in the section covered by the page

### Layouts

|Layout|Description|
|-|-|
|100 Scale|Map at 1:100 Scale|
|400 Scale|Map at 1:400 Scale|

## Process

1. Get cadastral features for selected year, with user input
2. Get tax map features
3. For each tax page:
    1. Set Tax Map page extent to page feature
    2. Set locators based on intersected PLSS features
    3. Adjust page surrounds to match page attributes
    4. Export to PDF

# Setup

## Variables for the Year

Set these variables to generate maps for the year.

In [31]:
tax_year = 2021
NAS_loc = "//172.20.2.98/gis/Historic Maps/Tax Map Pages"

## Modules

In [2]:
import pandas as pd
import arcpy
from arcgis import GIS
from arcgis.geometry import filters
from getpass import getpass
import time

## Log in to Portal, get Layers

In [3]:
gis = GIS('home')

tax_maps = gis.content.get('ee54052df6d64689b577daf1673933ad').layers[0]
plss = gis.content.get('39620d6f9566420aa1f258be185fbe95')
hosted_fabric = gis.content.get('647e966722f943d0b5d49f82288a839e')
hosted_boundaries = gis.content.get('f8111964fc964f70b0a6376366988185')

## Globals

We'll use these variables whether we use the interactive section or the batch export.

In [4]:
# Current Project
arpx = arcpy.mp.ArcGISProject('CURRENT')

# Main map
the_map = arpx.listMaps('Tax Map')[0]

# Tax map layers
pages = the_map.listLayers('Tax Map Pages')[0]
the_page = the_map.listLayers('Subject Page')[0]


## Layouts and locators
# 100 Scale layouts and frames
lyt_100 = arpx.listLayouts('100 Scale')[0]
twp_loc_100 = lyt_100.listElements('MAPFRAME_ELEMENT', 'Township Locator')[0]
sec_loc_100 = lyt_100.listElements('MAPFRAME_ELEMENT', 'Section Locator')[0]
pg_loc = lyt_100.listElements('MAPFRAME_ELEMENT', 'Page Locator')[0]

# 400 Scale layouts and frames
lyt_400 = arpx.listLayouts('400 Scale')[0]
twp_loc_400 = lyt_400.listElements('MAPFRAME_ELEMENT', 'Township Locator')[0]
sec_loc_400 = lyt_400.listElements('MAPFRAME_ELEMENT', 'Section Locator')[0]

# Locator Maps
twp_map = arpx.listMaps('Township Locator')[0]
sec_map = arpx.listMaps('Section Locator')[0]
pg_map = arpx.listMaps('Page Locator')[0]

# Locator Layers
sec_feats = sec_map.listLayers('Sections')[0]
twp_feats = twp_map.listLayers('Townships')[0]
pg_feats = pg_map.listLayers('Pages')[0]
sec_pages = sec_map.listLayers('Pages')[0]


## Layer lists
# Fabric Layers
parcels = hosted_fabric.layers[1]
subs = hosted_fabric.layers[2]
lots = hosted_fabric.layers[3]
corp = hosted_fabric.layers[7]

# Boundaries
p_lines = hosted_boundaries.layers[3]
l_lines = hosted_boundaries.layers[2]
s_lines = hosted_boundaries.layers[6]
c_lines = hosted_boundaries.layers[5]

# List for iterables
fabric_layers = [
    parcels,
    subs,
    lots,
    corp,
    p_lines,
    l_lines,
    s_lines,
    c_lines
]

## Functions

|Function|Description|
|-|-|
|`query_pages`|Filters tax map pages|
|`convert_extent`|Converts an extent from `arcgis` to an `arcpy.Extent` object|

In [5]:
def convert_extent(ext, buffer):
    """
    Converts an extent object retrieved from the arcgis python module to a an arcpy.Extent object.
    
        Parameters:
            ext (tuple): An extent from a geometry feature's extent property, formatted as (xmin, ymin, xmax, ymax)
            buffer (float): The amount to buffer the extent so that map features do not extend to edge, as a percent of the total.
        
        Returns:
            arcpy.Extent
    """
    x_min, y_min, x_max, y_max = ext

    hbuff = (x_max - x_min) * buffer
    vbuff = (y_max - y_min) * buffer

    return arcpy.Extent(x_min - hbuff,
                       y_min - vbuff,
                       x_max + hbuff,
                       y_max + vbuff)


def get_maps(arpx):
    """
    Prints a list of maps in the current project.
    
        Parameters:
            arpx (ArcGISProject): The current project.
            
        Returns:
            List of maps in current project.
    """
    print('Maps in Project:')
    for m in arpx.listMaps():
        print(f'  {arpx.listMaps().index(m):{3}} | {m.name}')

        
def get_layouts(arpx):
    """
    Prints a list of layouts in the current project.
    
        Parameters:
            arpx (ArcGISProject): The current project.
            
        Returns:
            List of layouts in current project.
    """
    print('Layouts in Project:')
    for l in arpx.listLayouts():
        print(f'  {arpx.listLayouts().index(l):{3}} | {l.name}')

        
def get_layers(some_map):
    """
    Prints a list of maps in the current project.
    
        Parameters:
            some_map (Map): A map in the project
            
        Returns:
            List of layers in selected map.
    """
    print(f'Layers in {some_map.name}:')
    for m in some_map.listLayers():
        print(f'  {some_map.listLayers().index(m):{3}} | {m.name}')
        
        
def label_filter(geom_filt):
    """
    Given a list of layers (defined in the globals above), set the Standard label class to include only those features with the relationship defined in `geom_filt`.
    
        Parameters:
            layer_list (list): A list of services layers defined in 
    """
    for layer in fabric_layers:
        
        layer_ids = layer.query(geometry_filter=geom_filt, return_ids_only=True)['objectIds']

        q_str = f"objectid in ({','.join([str(x) for x in layer_ids])})"

        for lclass in the_map.listLayers(layer.properties.name)[0].listLabelClasses():
            lclass.SQLQuery = q_str

In [30]:
lyrs = the_map.listLayers('Cadastral Stuff')[0].listLayers()

[l.definitionQuery for l in lyrs if not l.isGroupLayer]

["(created_date IS NULL Or created_date < timestamp '2022-01-08 00:00:00') And (retired_date IS NULL Or retired_date > timestamp '2022-01-08 00:00:00')", "(created_date IS NULL Or created_date < timestamp '2022-01-08 00:00:00') And (retired_date IS NULL Or retired_date > timestamp '2022-01-08 00:00:00')", "(created_date IS NULL Or created_date < timestamp '2022-01-08 00:00:00') And (retired_date IS NULL Or retired_date > timestamp '2022-01-08 00:00:00')", "(created_date IS NULL Or created_date < timestamp '2022-01-08 00:00:00') And (retired_date IS NULL Or retired_date > timestamp '2022-01-08 00:00:00')", "(created_date IS NULL Or created_date < timestamp '2022-01-08 00:00:00') And (retired_date IS NULL Or retired_date > timestamp '2022-01-08 00:00:00')", "(created_date IS NULL Or created_date < timestamp '2022-01-08 00:00:00') And (retired_date IS NULL Or retired_date > timestamp '2022-01-08 00:00:00')", "(created_date IS NULL Or created_date < timestamp '2022-01-08 00:00:00') And (re

## Prep Work

Get the tax maps into their dataframes.

In [5]:
t_100 = tax_maps.query(where="scale = '100'", as_df=True)
t_400 = tax_maps.query(where="scale = '400'", as_df=True)

,letter,scale,description,page,objectid,township,SHAPE
0,H,100,Oswego Township\n\nE ½ of SE ¼\nSection 25\n\n...,03-25H,1,03,"{""rings"": [[[1004393.8899819106, 1815326.53997..."
1,A,100,Oswego Township\n\nW ½ of NW ¼\nSection 36\n\n...,03-36A,2,03,"{""rings"": [[[999201.5831093304, 1812588.582200..."
2,B,100,Oswego Township\n\nE ½ of NW ¼\nSection 36\n\n...,03-36B,3,03,"{""rings"": [[[1001840.6299965791, 1812634.13985..."
3,E,100,Oswego Township\n\nW ½ of SW ¼\nSection 36\n\n...,03-36E,4,03,"{""rings"": [[[1000543.0000459105, 1809964.76789..."
4,F,100,Oswego Township\n\nE ½ of SW ¼\nSection 36\n\n...,03-36F,5,03,"{""rings"": [[[1001899.9399173297, 1809987.23044..."


Set definition queries on the cadastral features.

In [40]:
feat_query = f"(created_date IS NULL Or created_date < timestamp '{tax_year + 1}-01-08 00:00:00') AND (retired_date IS NULL Or retired_date > timestamp '{tax_year + 1}-01-08 00:00:00')"

for l in the_map.listLayers('Cadastral Stuff')[0].listLayers():
    if not l.isGroupLayer:
        l.definitionQuery = feat_query

# Demonstration

## 1:100 Scale Maps

We'll use the first map in the frame as an example, and walk through the process step by step so that it is well-documented and (hopefully) well-understood what's going on in this process.

In [6]:
test_page = t_100.loc[50, :]
test_page

letter                                                         A
scale                                                        100
description    Oswego Township\n\nW ½ of NW ¼\nSection 16\n\n...
page                                                      03-16A
objectid                                                      51
township                                                      03
SHAPE          {'rings': [[[982941.719959829, 1828215.3800146...
Name: 50, dtype: object

### Filtering Data

Using the separate dataframe, we can hold the subject page in memory and access its attributes, but make it disappear from the map itself. The remaining tax map page features will stay visible to screen out non-subject areas lightly.

In [11]:
# Set query for page layers
pages.definitionQuery = f"page <> '{test_page['page']}'"

the_page = the_map.listLayers('Subject Page')[0]
the_page.definitionQuery = f"page = '{test_page['page']}'"

### Setting Extent

We'll take that same subject page feature and use its `SHAPE` attribute to create an `arcpy.Extent` object, then pass that to the layout's `camera`.

In [10]:
pg_ext = convert_extent(test_page['SHAPE'].extent, 0.1)

lyt_100 = arpx.listLayouts('100 Scale')[0]

main_frame = lyt_100.listElements('MAPFRAME_ELEMENT', 'Main Frame')[0]

main_frame.camera.setExtent(pg_ext)

main_frame.camera.scale = 2400

### Updating Locators

You'd think something like this would be easier, but getting the locators to update the way we want is a bit of a process.

For the **Township Locator**, it's a simple as selecting the right feature. For the **Section Locator**, however, we need the locator map filtered to and centered around the township highlighted in the other locator.

For the 1:100 maps, we also want the **Page Locator** filtered down to the single section from the second locator.

To do this, we have will

1. Create a `geometry_filter` from the page shape's centroid
2. Pass that filter into queries to the locator features
3. Use the returned OIDs to
    1. Derive various extents
    2. Select and filter features

In [12]:
pg_cent_filt = filters.within(test_page['SHAPE'].true_centroid, test_page['SHAPE'].spatialReference)

# Townships
twp = plss.layers[0].query(geometry_filter = pg_cent_filt, as_df = True).iloc[0]

# Sections
sec = plss.layers[1].query(geometry_filter = pg_cent_filt, as_df = True).iloc[0]

Using the `objectid` from the returned PLSS features, we can update those maps!

In [38]:
# Select township
twp_feats.setSelectionSet([int(twp.objectid)])

In [36]:
# Select section, filter secs outside of twp; turn off pages if still on
sec_pages.visible = False
sec_pages.setSelectionSet([])
sec_feats.setSelectionSet([int(sec.objectid)])
sec_feats.definitionQuery = f"twpid = {twp['twp']}"

# Section locator to township extent
sec_ext_100 = convert_extent(twp['SHAPE'].extent, 0.05)
sec_loc_100.camera.setExtent(sec_ext_100)

In [35]:
# Select page, filter pages outside of sec
pg_feats.setSelectionSet([int(test_page.objectid)])
pg_feats.definitionQuery = f"scale = '100' and SUBSTRING(page, 4, 2) = '{sec['sect']}' and SUBSTRING(page, 2, 1) = '{twp['twp']}'" 

# Page locator to section extent
pg_ext = convert_extent(sec['SHAPE'].extent, 0.05)
pg_loc.camera.setExtent(pg_ext)

In [34]:
sec.objectid

297

### Feature Labels

For the nicest-possible display, we want to limit the number of features being labelled in the layout to those which are on the page itself, or just over the margin.

To do this, we'll use another **geometry filter** of the page feature, plus a slight buffer. We'll use the custom `label_filter` function to pass the filtered geometry OIDs to the label class definition.

In [16]:
pg_filt = filters.intersects(test_page['SHAPE'].buffer(25).densify('ANGLE', 10, 10), test_page['SHAPE'])

label_filter(pg_filt)

#### Parcels

For parcels in particular, we want to append the appropriate parcel subtype query to the SQL string, so we'll do that separately.

In [17]:
the_map.listLayers('Parcels')[0].listLabelClasses('Standard')[0].SQLQuery += " and parcel_type = 'Ownership Parcel'"
the_map.listLayers('Parcels')[0].listLabelClasses('Condos')[0].SQLQuery += " and parcel_type = 'Condominium Unit'"
the_map.listLayers('Parcels')[0].listLabelClasses('Leaseholds')[0].SQLQuery += " and parcel_type = 'Leasehold or Other'"

### Page Labels

Similarly, we want adjacent tax map pages labeled around the edge of this page.

In [18]:
p_ids = tax_maps.query(where="scale = '100'", geometry_filter=pg_filt, return_ids_only=True)['objectIds']

pages.listLabelClasses()[0].SQLQuery = f"objectid in ({','.join([str(x) for x in p_ids])})"

### Updating Text

On the tax map's margin, there are a few text elements that need to be updated, which we will do here.

In [19]:
lyt_100.listElements('TEXT_ELEMENT', 'Page Name')[0].text = test_page['page']
lyt_100.listElements('TEXT_ELEMENT', 'Page Description')[0].text = test_page['description'] + '\n\nKendall County, Illinois'

### Export

In [39]:
# Export layout item
lyt_100.exportToPDF(
    out_pdf = f"//172.20.2.98/gis/Historic Maps/Tax Map Pages/{test_page['page']}_2021_line.pdf",
    resolution = 300,
    image_quality = 'NORMAL',
    layers_attributes = 'NONE',
    georef_info = False
)

'//172.20.2.98/gis/Historic Maps/Tax Map Pages/03-16A_2021_line.pdf'

## 400 Scale Maps

Mostly the same as the 100 scale, but we won't worry about the page locator this time. We also make the pages themselves visible on the Section locator.

In [8]:
test_page = t_400.loc[30, :]

#Geometry Filter
pg_cent_filt = filters.within(test_page['SHAPE'].true_centroid, test_page['SHAPE'].spatialReference)

# Townships
twp = plss.layers[0].query(geometry_filter = pg_cent_filt, as_df = True).iloc[0]

# Sections
sec = plss.layers[1].query(geometry_filter = pg_cent_filt, as_df = True).iloc[0]

In [9]:
# Set query for page layers
pages.definitionQuery = f"page <> '{test_page['page']}' and scale <> '100'"

the_page.definitionQuery = f"page = '{test_page['page']}' and scale <> '100'"

In [ ]:
# Main Frame extent and scale
pg_ext = convert_extent(test_page['SHAPE'].extent, 0.1)

lyt_400.listElements('MAPFRAME_ELEMENT', 'Main Frame')[0].camera.setExtent(pg_ext)
lyt_400.listElements('MAPFRAME_ELEMENT', 'Main Frame')[0].camera.scale = 9600

### Locators

In [22]:
# Select township
twp_feats.setSelectionSet([twp['objectid']])

# Turn on pages layer
sec_pages.visible = True

# Select page, filter secs outside of twp, clear section selection if any
sec_pages.setSelectionSet([test_page['objectid']])
sec_feats.setSelectionSet([])
sec_feats.definitionQuery = f"twpid = {twp['twp']}"

# Section locator to township extent
sec_ext_400 = convert_extent(twp['SHAPE'].extent, 0.05)
sec_loc_400.camera.setExtent(sec_ext_400)

### Labels

In [25]:
pg_filt = filters.intersects(test_page['SHAPE'].buffer(25).densify('ANGLE', 10, 10), test_page['SHAPE'])

label_filter(pg_filt)

the_map.listLayers('Parcels')[0].listLabelClasses('Standard')[0].SQLQuery += " and parcel_type = 'Ownership Parcel'"
the_map.listLayers('Parcels')[0].listLabelClasses('Condos')[0].SQLQuery += " and parcel_type = 'Condominium Unit'"
the_map.listLayers('Parcels')[0].listLabelClasses('Leaseholds')[0].SQLQuery += " and parcel_type = 'Leasehold or Other'"

p_ids = tax_maps.query(where="scale = '400'", geometry_filter=pg_filt, return_ids_only=True)['objectIds']

pages.listLabelClasses()[0].SQLQuery = f"objectid in ({','.join([str(x) for x in p_ids])})"

### Text Elements

In [26]:
lyt_400.listElements('TEXT_ELEMENT', 'Page Name')[0].text = test_page['page']
lyt_400.listElements('TEXT_ELEMENT', 'Page Description')[0].text = test_page['description'] + '\n\nKendall County, Illinois'

### Export

In [27]:
# Export layout item
lyt_400.exportToPDF(
    out_pdf = f"//172.20.2.107/gis/Historic Maps/Tax Map Pages/{test_page['page']}_2020_line.pdf",
    resolution = 300,
    image_quality = 'NORMAL',
    layers_attributes = 'NONE',
    georef_info = False
)

'//172.20.2.107/gis/Historic Maps/Tax Map Pages/08-C_2020_line.pdf'

# Batch Export

Now we'll do a bunch of maps at once!

In [18]:
# 100 Scale
start = time.perf_counter()

n = 180
while n < len(t_100):
    
    # Get page feature
    pg = t_100.loc[n, :]
    
    # Set page extent and scale
    pg_ext = convert_extent(pg['SHAPE'].extent, 0.1)

    lyt_100.listElements('MAPFRAME_ELEMENT', 'Main Frame')[0].camera.setExtent(pg_ext)
    lyt_100.listElements('MAPFRAME_ELEMENT', 'Main Frame')[0].camera.scale = 2400
    
    # Set query for page layers
    pages.definitionQuery = f"page <> '{pg['page']}'"
    the_page.definitionQuery = f"page = '{pg['page']}'"
    
    # Geometry filter for PLSS
    pg_cent_filt = filters.within(pg['SHAPE'].true_centroid, pg['SHAPE'].spatialReference)

    # Townships
    twp = plss.layers[0].query(geometry_filter = pg_cent_filt, as_df = True).iloc[0]

    # Sections
    sec = plss.layers[1].query(geometry_filter = pg_cent_filt, as_df = True).iloc[0]
    
    
    ## Locators
    # Select township
    twp_feats.setSelectionSet([int(twp['objectid'])])
    
    # Select section, filter secs outside of twp; turn off pages if still on
    sec_pages.visible = False
    sec_pages.setSelectionSet([])
    sec_feats.setSelectionSet([int(sec['objectid'])])
    sec_feats.definitionQuery = f"twpid = {twp['twp']}"

    # Section locator to township extent
    sec_ext_100 = convert_extent(twp['SHAPE'].extent, 0.05)
    sec_loc_100.camera.setExtent(sec_ext_100)
    
    # Select page, filter pages outside of sec
    pg_feats.setSelectionSet([int(pg['objectid'])])
    pg_feats.definitionQuery = f"scale = '100' and SUBSTRING(page, 4, 2) = '{sec['sect']}' and SUBSTRING(page, 2, 1) = '{twp['twp']}'" 

    # Page locator to section extent
    pg_ext = convert_extent(sec['SHAPE'].extent, 0.05)
    pg_loc.camera.setExtent(pg_ext)
    
    ## Labels
    # Features
    pg_filt = filters.intersects(pg['SHAPE'].buffer(25).densify('ANGLE', 10, 10), pg['SHAPE'])
    
    label_filter(pg_filt)
    
    # Append subtypes to parcel filters
    the_map.listLayers('Parcels')[0].listLabelClasses('Standard')[0].SQLQuery += " and parcel_type = 'Ownership Parcel'"
    the_map.listLayers('Parcels')[0].listLabelClasses('Condos')[0].SQLQuery += " and parcel_type = 'Condominium Unit'"
    the_map.listLayers('Parcels')[0].listLabelClasses('Leaseholds')[0].SQLQuery += " and parcel_type = 'Leasehold or Other'"
    
    # Pages
    p_ids = tax_maps.query(where="scale = '100'", geometry_filter=pg_filt, return_ids_only=True)['objectIds']

    pages.listLabelClasses()[0].SQLQuery = f"objectid in ({','.join([str(x) for x in p_ids])})"
    
    ## Update Text
    lyt_100.listElements('TEXT_ELEMENT', 'Page Name')[0].text = pg['page']
    lyt_100.listElements('TEXT_ELEMENT', 'Page Description')[0].text = pg['description'] + '\n\nKendall County, Illinois'
    
    ## Export
    lyt_100.exportToPDF(
        out_pdf = f"{NAS_loc}/{pg['page']}_{tax_year}_line.pdf",
        resolution = 300,
        image_quality = 'NORMAL',
        layers_attributes = 'NONE',
        georef_info = False
    )
    
    if (n % 10) == 0 and n > 0:
        elapsed = (time.perf_counter() - start)
        remaining = (time.perf_counter() - start)/(n / len(t_100)) - elapsed
        print(f'{int(n / len(t_100) * 100)//1:3} %  |  {n:3} of {len(t_100):3} printed.  |  {elapsed/60:>3.0f} min elapsed, est. {remaining/60:>3.0f} remaining')
    
    n += 1
    
print(f'Finished at {(time.perf_counter() - start)/60:.0f} minutes.')

 28 %  |  180 of 638 printed.  |    0 min elapsed, est.   1 remaining
 29 %  |  190 of 638 printed.  |    2 min elapsed, est.   4 remaining
 31 %  |  200 of 638 printed.  |    3 min elapsed, est.   7 remaining
 32 %  |  210 of 638 printed.  |    4 min elapsed, est.   9 remaining
 34 %  |  220 of 638 printed.  |    6 min elapsed, est.  11 remaining
 36 %  |  230 of 638 printed.  |    7 min elapsed, est.  13 remaining
 37 %  |  240 of 638 printed.  |    8 min elapsed, est.  14 remaining
 39 %  |  250 of 638 printed.  |   10 min elapsed, est.  15 remaining
 40 %  |  260 of 638 printed.  |   11 min elapsed, est.  16 remaining
 42 %  |  270 of 638 printed.  |   12 min elapsed, est.  17 remaining
 43 %  |  280 of 638 printed.  |   14 min elapsed, est.  18 remaining
 45 %  |  290 of 638 printed.  |   15 min elapsed, est.  18 remaining
 47 %  |  300 of 638 printed.  |   18 min elapsed, est.  20 remaining
 48 %  |  310 of 638 printed.  |   20 min elapsed, est.  21 remaining
 50 %  |  320 of 638

In [19]:
### 400 Scale

start = time.perf_counter()

n = 0
while n < len(t_400):
    
    # Page feature
    pg = t_400.iloc[n, :]
    
    #Geometry Filter
    pg_cent_filt = filters.within(pg['SHAPE'].true_centroid, pg['SHAPE'].spatialReference)

    # Townships
    twp = plss.layers[0].query(geometry_filter = pg_cent_filt, as_df = True).iloc[0]

    # Sections
    sec = plss.layers[1].query(geometry_filter = pg_cent_filt, as_df = True).iloc[0]
    
    # Set query for page layers
    pages.definitionQuery = f"page <> '{pg['page']}' and scale <> '100'"

    the_page.definitionQuery = f"page = '{pg['page']}' and scale <> '100'"
    
    # Main Frame extent and scale
    pg_ext = convert_extent(pg['SHAPE'].extent, 0.1)

    lyt_400.listElements('MAPFRAME_ELEMENT', 'Main Frame')[0].camera.setExtent(pg_ext)
    lyt_400.listElements('MAPFRAME_ELEMENT', 'Main Frame')[0].camera.scale = 9600
    
    ## Locators
    # Select township
    twp_feats.setSelectionSet([int(twp['objectid'])])

    # Turn on pages layer
    sec_pages.visible = True

    # Select page, filter secs outside of twp, clear section selection if any
    sec_pages.setSelectionSet([int(pg['objectid'])])
    sec_feats.setSelectionSet([])
    sec_feats.definitionQuery = f"twpid = {twp['twp']}"

    # Section locator to township extent
    sec_ext_400 = convert_extent(twp['SHAPE'].extent, 0.05)
    sec_loc_400.camera.setExtent(sec_ext_400)
    
    ## Feature Labeling
    pg_filt = filters.intersects(pg['SHAPE'].buffer(25).densify('ANGLE', 10, 10), pg['SHAPE'])

    label_filter(pg_filt)

    the_map.listLayers('Parcels')[0].listLabelClasses('Standard')[0].SQLQuery += " and parcel_type = 'Ownership Parcel'"
    the_map.listLayers('Parcels')[0].listLabelClasses('Condos')[0].SQLQuery += " and parcel_type = 'Condominium Unit'"
    the_map.listLayers('Parcels')[0].listLabelClasses('Leaseholds')[0].SQLQuery += " and parcel_type = 'Leasehold or Other'"

    p_ids = tax_maps.query(where="scale = '400'", geometry_filter=pg_filt, return_ids_only=True)['objectIds']

    pages.listLabelClasses()[0].SQLQuery = f"objectid in ({','.join([str(x) for x in p_ids])})"
    
    lyt_400.listElements('TEXT_ELEMENT', 'Page Name')[0].text = pg['page']
    lyt_400.listElements('TEXT_ELEMENT', 'Page Description')[0].text = pg['description'] + '\n\nKendall County, Illinois'
    
    # Export layout item
    lyt_400.exportToPDF(
        out_pdf = f"{NAS_loc}/{pg['page']}_{tax_year}_line.pdf",
        resolution = 300,
        image_quality = 'NORMAL',
        layers_attributes = 'NONE',
        georef_info = False
    )
    
    if (n % 10) == 0 and n > 0:
        elapsed = (time.perf_counter() - start)
        remaining = (time.perf_counter() - start)/(n / len(t_400)) - elapsed
        print(f'{int(n / len(t_400) * 100)//1:3} %  |  {n:3} of {len(t_400):3} printed.  |  {elapsed/60:>3.0f} min elapsed, est. {remaining/60:>3.0f} remaining')
    
    n += 1
    
print(f'Finished at {(time.perf_counter() - start)/60:.0f} minutes.')

  6 %  |   10 of 162 printed.  |    2 min elapsed, est.  37 remaining
 12 %  |   20 of 162 printed.  |    4 min elapsed, est.  29 remaining
 18 %  |   30 of 162 printed.  |    6 min elapsed, est.  25 remaining
 24 %  |   40 of 162 printed.  |    8 min elapsed, est.  23 remaining
 30 %  |   50 of 162 printed.  |   10 min elapsed, est.  22 remaining
 37 %  |   60 of 162 printed.  |   12 min elapsed, est.  20 remaining
 43 %  |   70 of 162 printed.  |   14 min elapsed, est.  18 remaining
 49 %  |   80 of 162 printed.  |   19 min elapsed, est.  19 remaining
 55 %  |   90 of 162 printed.  |   22 min elapsed, est.  17 remaining
 61 %  |  100 of 162 printed.  |   27 min elapsed, est.  17 remaining
 67 %  |  110 of 162 printed.  |   38 min elapsed, est.  18 remaining
 74 %  |  120 of 162 printed.  |   44 min elapsed, est.  15 remaining
 80 %  |  130 of 162 printed.  |   60 min elapsed, est.  15 remaining
 86 %  |  140 of 162 printed.  |   63 min elapsed, est.  10 remaining
 92 %  |  150 of 162